# Final Evaluation 

In this notebook the final resut is going to be evaluated. The data is first loaded into the Notebokk and then loaded into the current database.
After that the pipeline in the scope of this thesis is applied and the results collected for final evaluation

In [1]:
import pandas as pd
import numpy
import re
#path="Data/mercari-price-suggestion-challenge/test.tsv"
path="Data/mercari-price-suggestion-challenge/test_sample.tsv"
#Read the file
data=pd.read_csv(path,sep="\t" )
print(f"The number of unique categories is {data['category_name'].nunique()}")


data = data[["name", "category_name"]]
data=data[1:5]
data



#FOR creation
# sampled_data=data.sample(n=500, random_state=42)
# # Optional: check the shape to confirm
# print(sampled_data.shape)

# # Optional: preview the sampled data
# sampled_data.head()
# # Write the sampled data back to a TSV file
# sampled_data.to_csv("Data/mercari-price-suggestion-challenge/test_sample.tsv", sep='\t', index=False)

# print("Sample saved successfully to test_sample.tsv")

The number of unique categories is 199


,name,category_name
1,City color Be Matte Blush in Hibiscus,Beauty/Makeup/Face
2,Pageant Dress Slip,Kids/Girls (4+)/Dresses
3,New! VS Pink Bikini Top,Women/Swimwear/Two-Piece
4,Bring me the horizon posters,Home/Artwork/Posters & Prints


In [2]:
unique_categories = data['category_name'].unique()

unique_categories=unique_categories[0]
temp_df=pd.DataFrame()
temp_df=data[data['category_name']==unique_categories]

temp_df

,name,category_name
1,City color Be Matte Blush in Hibiscus,Beauty/Makeup/Face


In [3]:
from Evaluation.evaluation import evaluate_results
from Main.combined_pipeline import combined_pipeline
from Evaluation.test_evaluation import evaluate_results
from Utilities.database import query_database

/home/vlapan/Documents/Masterarbeit/Relational/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Function for populating the database
def create_and_populate_table(dataframe, table_name):
    try:
        # Clean table name (optional)
        table_name = table_name.replace(" ", "_")

        # Drop table if it exists
        delete_table_query = f"DROP TABLE IF EXISTS {table_name} CASCADE;"
        query_database(delete_table_query, printing=False)

        # Convert all data to string format
        dataframe = dataframe.astype(str)

        # Create table with a single column
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (Name TEXT , Category Text);"
        query_database(create_table_query, printing=False)
        print(f"Table '{table_name}' created successfully.")

        values = []
        for _, row in dataframe.iterrows():
            name = row['name'].replace("'", "''")  # Escape single quotes in SQL
            category = row['category_name'].replace("'", "''")  # Escape single quotes in SQL
            values.append(f"('{name}', '{category}')")

        if values:
            insert_query = f"INSERT INTO {table_name} VALUES {', '.join(values)};"
            query_database(insert_query, printing=False)
            print(f"Table '{table_name}' populated successfully.")
        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [5]:
create_and_populate_table(data, "mercari_data")

Table 'mercari_data' created successfully.
Table 'mercari_data' populated successfully.


In [6]:
query=f"SELECT * FROM mercari_data WHERE Category='{unique_categories}'"
expected=query_database(query)

The final answer to the query is [('City color Be Matte Blush in Hibiscus', 'Beauty/Makeup/Face')]


In [7]:
pipeline_query=f"SELECT * FROM mercari_data WHERE mercari_data.name='{unique_categories}'"
actual=combined_pipeline(query=None,initial_sql_query=pipeline_query,aux=True)

The final answer to the query is []
--------------------------------------
The 
”SELECT * FROM mercari_data WHERE mercari_data.name='Beauty/Makeup/Face'
 has a WHERE clause.
<bound method TokenList.get_name of <Where 'WHERE ...' at 0x7EB4A6827350>>
The final answer to the query is [('City color Be Matte Blush in Hibiscus',), ('Pageant Dress Slip',), ('New! VS Pink Bikini Top',), ('Bring me the horizon posters',)]
The phrase is:
 has the same meaning as (also in another language) or is the same as. 
temp_string: Beauty/Makeup/Face
temp_list: [('City color Be Matte Blush in Hibiscus',), ('Pageant Dress Slip',), ('New! VS Pink Bikini Top',), ('Bring me the horizon posters',)]
The semantics dic is {"WHERE mercari_data.name='Beauty/Makeup/Face'_comparison_Beauty/Makeup/Face": {'Beauty/Makeup/Face': []}}
An error occurred: syntax error at or near "/"
LINE 1: DROP TABLE IF EXISTS WHEREmercari_datanameBeauty/Makeup/Face...
                                                        ^

The final pr

In [ ]:


accuracy, precision, recall, f1_score = evaluate_results(data, temp_df)